<a href="https://colab.research.google.com/github/EltondeFreitas/projeto_imersao_alura_google/blob/main/ProjetoChatBotAjudaParaEnchentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chamando a biblioteca google-generativeai para trabalhar com IA

In [89]:
!pip install -q -U google-generativeai

In [90]:
import numpy as np #  suporta o processamento de grandes, multi-dimensionais arranjos e matrizes (calculos)
import pandas as pd # manipulação e análise de dados (tabelas e planilhas com linha e colunas)
import google.generativeai as genai # Crie mecanismos de pesquisa aprimorados ou IA para melhorar a experiência do cliente
import json # Manipulação de arquivos JSON

Define a chave api do google e inicialisa ela, lembrando que ela precisa estar habilitada em Secrets em seu notebook

In [91]:
GOOGLE_API_KEY = GOOGLE_API_KEY
genai.configure(api_key=GOOGLE_API_KEY)

Vamos usar Embeddings, para representar as palavras em tokens, para criar uma representação vetorial, entender o contexto, ele compara os tokens em questão de proximidade entre os embeddings, os tokens, para ver o que está mais próximo

In [92]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


Pegando o arquivo JSON (enchentes.json) com todas as respostas validadas para ajudar a quem enfrentou enchente, a vantagem desse arquivo é que são respostas retiradas de manuais sobre enchente e o modelo não vai alucinar. Esse arquivo ainda pode receber atualizações.

In [93]:
# Pegando o arquivo JSON com repostas para lidar com enchentes, esse arquivo vai estar no meu google drive
with open('/content/drive/MyDrive/enchentes.json') as f:
  dados_json = json.load(f)

# Extraindo os dados das chaves "Título" e "Conteúdo"
titulos = []
conteudos = []

for item in dados_json:
  titulos.append(item['Titulo'])
  conteudos.append(item['Conteúdo'])

# Criando o DataFrame com as novas colunas
df = pd.DataFrame({
  'Título': titulos,
  'Conteúdo': conteudos
})

df

,Título,Conteúdo
0,Plano de Emergência Familiar,Crie um plano de emergência com sua família. D...
1,Kit de Emergência para Enchentes,Monte um kit de suprimentos para emergências c...
2,Mantenha-se Informado,Acompanhe as previsões meteorológicas e alerta...
3,Proteja sua Casa,"Eleve móveis, eletrodomésticos e objetos de va..."
4,Desligue a Eletricidade,Desligue a energia elétrica da sua casa se hou...
...,...,...
103,Proteja Documentos Importantes da Escola,Desenvolva um plano para proteger os documento...
104,Treinamento Básico de Primeiros Socorros para ...,Considere treinar funcionários responsáveis pe...
105,Utilize Alto-falantes e Sirenes para Alerta,Utilize alto-falantes e sirenes para alertar a...
106,Identifique Abrigos Temporários Potenciais,Identifique abrigos temporários potenciais na ...


Definindo nome para as colunas do DataFRame

In [94]:
df.columns = ["Titulo","Conteudo"]
df

,Titulo,Conteudo
0,Plano de Emergência Familiar,Crie um plano de emergência com sua família. D...
1,Kit de Emergência para Enchentes,Monte um kit de suprimentos para emergências c...
2,Mantenha-se Informado,Acompanhe as previsões meteorológicas e alerta...
3,Proteja sua Casa,"Eleve móveis, eletrodomésticos e objetos de va..."
4,Desligue a Eletricidade,Desligue a energia elétrica da sua casa se hou...
...,...,...
103,Proteja Documentos Importantes da Escola,Desenvolva um plano para proteger os documento...
104,Treinamento Básico de Primeiros Socorros para ...,Considere treinar funcionários responsáveis pe...
105,Utilize Alto-falantes e Sirenes para Alerta,Utilize alto-falantes e sirenes para alertar a...
106,Identifique Abrigos Temporários Potenciais,Identifique abrigos temporários potenciais na ...


Definindo o modelo que iremos utilizar: "models/embedding-001"

In [95]:
model = "models/embedding-001"

Converter em embeddings os dados obitidos no arquivo JSON

In [96]:
def embed_fn(title,text):
  return genai.embed_content(
      model=model,
      content=text,
      title=title,
      task_type="RETRIEVAL_DOCUMENT"
  )["embedding"] # resposta do gemini, tem que ser embedding, só queremos a lista de números

In [97]:
# nova coluna dentro do dataframe
# vai aplicar a função embed_fn em cada linha do dataframe (como se fosse um pequeno for usando lambda)
df["embedding"] = df.apply(lambda row: embed_fn(row["Titulo"],row["Conteudo"]),axis=1) # axis é para acessar as colunas
df

,Titulo,Conteudo,embedding
0,Plano de Emergência Familiar,Crie um plano de emergência com sua família. D...,"[-0.01407229, -0.018316519, -0.057802938, 0.02..."
1,Kit de Emergência para Enchentes,Monte um kit de suprimentos para emergências c...,"[-0.027788233, -0.004128557, -0.052528836, -0...."
2,Mantenha-se Informado,Acompanhe as previsões meteorológicas e alerta...,"[0.017145162, -0.02557839, -0.03672759, 0.0013..."
3,Proteja sua Casa,"Eleve móveis, eletrodomésticos e objetos de va...","[-0.016951056, -0.011099305, -0.015614939, 0.0..."
4,Desligue a Eletricidade,Desligue a energia elétrica da sua casa se hou...,"[-0.021782078, 0.009760111, -0.015626114, 0.02..."
...,...,...,...
103,Proteja Documentos Importantes da Escola,Desenvolva um plano para proteger os documento...,"[-0.0045299293, 0.0076391553, -0.02956098, 0.0..."
104,Treinamento Básico de Primeiros Socorros para ...,Considere treinar funcionários responsáveis pe...,"[0.005572392, -0.010920319, -0.032150466, 0.02..."
105,Utilize Alto-falantes e Sirenes para Alerta,Utilize alto-falantes e sirenes para alertar a...,"[0.026418833, -0.04777515, -0.038389895, 0.052..."
106,Identifique Abrigos Temporários Potenciais,Identifique abrigos temporários potenciais na ...,"[0.015998682, -0.006486429, -0.06227124, 0.035..."


Criando uma função para gerar e buscar a consulta na base de dados

In [98]:
def gerar_e_buscar_consulta(consulta,base,model):
  # crio o embedding da minha consulta
  embedding_da_consulta = genai.embed_content(
      model=model,
      content=consulta,
      task_type="RETRIEVAL_QUERY")["embedding"]
  #faço uma operação matemática para comparar minha consulta com o que tenho na minha base
  # np.dot multiplica a consulta pela base para fazer a comparação, o np.stack empilha os dados da coluna
  produtos_escalares = np.dot(np.stack(df["embedding"]),embedding_da_consulta)
  # verifica quem é o maior idnice, o mais proximo do que queremos buscar
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]


As respostas não são criativas, sempre serão as mesmas para a pergunta, isso nos dá a segurança do modelo não alucinar e gerar algo perigos que alguém aplique no caso de uma enchente.

In [101]:
consulta = input('O que deseja saber para lidar com enchentes?\n')

while consulta != "fim":
  trecho = gerar_e_buscar_consulta(consulta,df,model)

  model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config, safety_settings=safety_settings)
  response = model_2.generate_content(consulta)
  print(response.text)
  consulta = input('\n\n\nO que deseja saber para lidar com enchentes?\n')



O que deseja saber para lidar com enchentes?
fim
